In [1]:
import artm
from tqdm import tqdm
import os
import codecs

plus = 'booksX2'
model_path = 'models/'+plus+'/'
topics, min_df = 300, 1    
batch_size = 1000

def linkBatches(path,wv_file,batch_folder='batches',batches=None):
    if os.path.exists(path+batch_folder):
        # создание класса, работающего с батчами (уже заполненного)
        batch_vectorizer = artm.BatchVectorizer(data_path=path+batch_folder, data_format='batches', batches = batches)
    else:
        # создание класса, работающего с батчами и заполнение ими папки folder+'-'+str(batch_size)
        batch_vectorizer = artm.BatchVectorizer(data_path=path+wv_file,
                                                data_format="vowpal_wabbit",
                                                target_folder=path+batch_folder,
                                                batch_size=batch_size,
                                                batches=batches)
    return batch_vectorizer
# инициализация модели
def doInit(T_init=300, reuse_theta=False, cache_theta=False):
    global T, model_artm, topic_names
    T = T_init # количество тем
    topic_names = ["sbj"+str(i) for i in range(T)]
    model_artm = artm.ARTM(num_topics=T, topic_names=topic_names,
                           class_ids={"text": 1,"title": 3,"desc": 1.5,"lang": 0}, #"keys": 0.5,
                           num_document_passes=2, reuse_theta=reuse_theta, cache_theta=cache_theta, seed=-1,
                           num_processors = 6)

def loadModel(topics):
    doInit(topics)
    model_artm.load(model_path+'model'+str(topics)+'_'+str(min_df)+'.dat')

In [2]:
def b(j):
    n1 = j%26
    n2 = (j-n1)%26**2/26
    n3 = (j-n1-n2*26)%26**4/26**2
    return (chr(n3+ord_a)+chr(n2+ord_a)+chr(n1+ord_a)) 
def g(n,s):
    return n/s + (1 if n%s > 0 else 0)

def doBatches(path,file_,lst=None):
    batches = None if lst == None else ['aaa'+b(j)+'.batch' for j in lst]
    #print(batches)
    batch_vectorizer = linkBatches(path,file_+'.wv','batches',batches)
    theta = model_artm.transform(batch_vectorizer)
    nCount = len(theta.columns) 
    for i in tqdm(range(nAll,nAll+nCount)):
        vals = theta[i+1].values
        lst = urls[i]
        for j in range(topics):
            if vals[j] >= 0.01:
                lst += u' %s^%s' % (j,vals[j])
        fw.write(lst+'\n')
    return nCount

def doTems(path,file_,dt=0):
    global ord_a, nAll, fw
    fw, nAll = codecs.open(path+plus+'-'+str(topics)+'_'+str(min_df)+'.tems','w', encoding="utf8"), 0
    if dt == 0:
        doBatches(path,file_)
    else:
        ord_a, nRecs = ord('a'), len(urls)
        lst, n = [], 0
        for i in range(g(nRecs,batch_size)):
            if n == dt:
                nAll += doBatches(path,file_,lst)
                lst, n = [], 0
            lst.append(i)
            n += 1
        if len(lst) > 0: doBatches(path,file_,lst)
    fw.close()    
loadModel(topics)

In [3]:
def getName(name):
    return ', '.join(name.split(' |author ')[1].split(' |title '))+'|'
urls, path, file_ = [], 'models/books/','books'
with codecs.open(path+file_+'.wv', 'r', encoding="utf8") as f:
    for line in tqdm(f):
        name = getName(line.strip().split(' |text ')[0])
        urls.append(name)    
doTems(path,file_)

472it [00:00, 563.21it/s]
100%|██████████| 472/472 [00:00<00:00, 2240.27it/s]


In [4]:
num_tokens = 10
model_artm.scores.add(artm.TopTokensScore(name="top_words", num_tokens=num_tokens, class_id="text"))
scores = model_artm.get_score("top_words")

In [5]:
with codecs.open(path+plus+'-'+str(topics)+'_'+str(min_df)+'.topics', 'w', encoding="utf8") as f:
    for n in range(topics):
        words = []
        for i in range(num_tokens):
            words.append(scores.token[n*10+i])
        words = ' '.join(words)
        print 'tem'+str(n)+':',words
        f.write(str(n)+' '+words+'\n')

tem0: теодор присяжный борсный казнь буа теодора раш сапожник трибунал подсудимый
tem1: старшина окоп сотник мирон григорьевич сват хуандо гимнастерка джеймс астахов
tem2: лейтенант маруха охранник файнхальс марина пленница плен похититель эрнандо криб
tem3: стена двор грудь небо ворота ветер сон входить плечо сюда
tem4: официант марсенд нэнси милан осси филиппидес сампайо этторе памплона коимбра
tem5: письмо вставать приезжать рано написать уезжать бумага просить прочитывать конверт
tem6: тетрадь женевьева шарль гюбер филя пат тамбур гарриет приключение швейцер
tem7: томас консул сенатор будденброк сенат кот фауна христианин иоганн дора
tem8: аликс баден монархия николаша орестовна курортник гесс клементьев андозерская монарх
tem9: роджер мари флем кэтрин манфред юла монтгомери уорд вица гровер
tem10: носилки андреас жандарм ефрем белобрысый гринхусен небритый гринхусный туляречито элисабет
tem11: дон сеньорита раймундо танис кнут гамусо гауденсио рамона камило бенисья
tem12: джордж р

In [6]:
n = 0
tems0 = dict()
with codecs.open(path+plus+'-'+str(topics)+'_'+str(min_df)+'.tems','r', encoding="utf8") as f:
    for line in tqdm(f):
        vals = line.strip().split('| ')
        name = vals[0]
        tems = sorted([x.split('^') for x in vals[1].split(' ')],key=lambda f: (-float(f[1]),int(f[0])))
        for tem,weight in tems:
            x, y = int(tem), float(weight)
            z = tems0.get(x,[0,0.,dict()])
            z[0] += 1
            z[1] += y
            #z[2][name] = y 
            tems0[x] = z
        print name
        print tems[:3]
        n += 1

472it [00:00, 7999.58it/s]

Лагерквист_Пер, Пилигрим_в_море
[[u'139', u'0.0398054'], [u'256', u'0.0307695'], [u'133', u'0.0284788']]
Лагерквист_Пер, Варавва
[[u'169', u'0.0489056'], [u'228', u'0.0334308'], [u'164', u'0.027652']]
Лагерквист_Пер, Смерть_Агасфера
[[u'150', u'0.0341648'], [u'133', u'0.0330508'], [u'228', u'0.0326377']]
Лагерквист_Пер, Улыбка_вечности
[[u'166', u'0.0273724'], [u'293', u'0.0259491'], [u'210', u'0.0256855']]
Лагерквист_Пер, Святая_земля
[[u'133', u'0.0391276'], [u'135', u'0.0373494'], [u'228', u'0.0316627']]
Лагерквист_Пер, Карлик
[[u'139', u'0.0312809'], [u'129', u'0.0301227'], [u'150', u'0.0248563']]
Лагерквист_Пер, Сивилла
[[u'105', u'0.0393206'], [u'131', u'0.0391288'], [u'66', u'0.0320486']]
Лагерквист_Пер, В_мире_гость
[[u'3', u'0.0366707'], [u'166', u'0.0330973'], [u'143', u'0.0327244']]
Шойинка_Воле, Лес_тысячи_духов
[[u'206', u'0.0606486'], [u'152', u'0.0379074'], [u'163', u'0.0277265']]
Беккет_Сэмюэль, Мэлон_умирает
[[u'150', u'0.0321533'], [u'112', u'0.0291923'], [u'290', u'

In [7]:
sorted(tems0.items(),key=lambda f: (-f[1][0],f[1][1]))

[(150, [448, 9.455801999999998, {}]),
 (112, [398, 7.088542799999999, {}]),
 (238, [392, 7.281962400000008, {}]),
 (135, [387, 6.240941100000006, {}]),
 (81, [385, 5.782209099999999, {}]),
 (228, [342, 5.340344699999996, {}]),
 (3, [338, 6.063905099999997, {}]),
 (210, [309, 4.334047299999999, {}]),
 (36, [302, 4.840629100000002, {}]),
 (161, [299, 4.510281100000001, {}]),
 (45, [298, 4.1410765, {}]),
 (217, [298, 4.745310700000006, {}]),
 (288, [294, 5.164258700000003, {}]),
 (139, [285, 4.690374900000005, {}]),
 (148, [277, 3.6378061, {}]),
 (66, [275, 4.5172592999999965, {}]),
 (293, [273, 4.6900449, {}]),
 (122, [265, 3.5728359999999997, {}]),
 (211, [265, 4.183015099999999, {}]),
 (140, [265, 4.346282099999995, {}]),
 (166, [263, 5.302201499999999, {}]),
 (234, [261, 4.150665900000001, {}]),
 (247, [255, 4.158273900000001, {}]),
 (25, [245, 3.4827423000000013, {}]),
 (241, [240, 3.4716709999999997, {}]),
 (137, [236, 3.8814713000000003, {}]),
 (164, [232, 3.744726899999999, {}]),
